In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas_profiling import ProfileReport
import sweetviz as sv
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
#למידת מכונה
from sklearn.model_selection import train_test_split
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df= pd.read_csv("5.4 full_data.csv")

In [ ]:
df.head()

# הוצאת וסידור משתנים


In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df['sw_pregnant'].fillna(value=0,inplace = True)

In [ ]:
df['sw_pregnant'].isnull().sum()

In [ ]:
for i in range(len(df['sw_pregnant'])):
    if df['sw_pregnant'][i]==2 or df['sw_pregnant'][i]==3:
        df['sw_pregnant'][i]=0
    elif df['sw_smoke_100'][i]==7 or df['sw_smoke_100'][i]==9:
        df['sw_smoke_100'][i]=2
    elif df['sw_smoke_actual'][i]==7:
        df['sw_smoke_actual'][i]=2

In [ ]:
df['sw_pregnant'].value_counts()

In [ ]:
df['sw_smoke_100'].value_counts()

In [ ]:
df['sw_smoke_actual'].value_counts()

צריך לחלק את הדאטה לאימון וטסט לפני שמתחילים לעבוד על הדטאה


In [ ]:
x_train , x_test = train_test_split(df,test_size=0.2,random_state=42)

בדיקות קשר

In [ ]:
stats.spearmanr(df['HOUSE_INDIV'], df['FAMILY_SIZE']) 

In [ ]:
cor_matrix=df.corr(method="spearman")

In [ ]:
plt.figure(figsize=(10,10))
ax =sns.heatmap(cor_matrix)

In [ ]:
cor_matrix['HOUSE_INDIV']['FAMILY_SIZE']

הוצאה לטבלה אחת ברורה כל הקורילציות הגדולות מ0.7 חיובי ושלילי


In [ ]:
#cor_matrix.dropna(inplace=True)
list_x=list()
list_y=list()
list_cor=list()
for x in cor_matrix.columns:
    for y in cor_matrix.columns:
        if abs(cor_matrix[x][y])>0.7 and x!=y:
           list_x.append(x)
           list_y.append(y)
           list_cor.append(cor_matrix[x][y])

In [ ]:
cor=pd.DataFrame()
cor['x']=list_x
cor['y']=list_y
cor['cor_value']=list_cor


In [ ]:
df_New=pd.DataFrame()
df_New['GLUCOSE']=df['GLUCOSE']
df_New['Glucose_basal']=df['Glucose_basal']
df_New.dropna(inplace=True)
print(df_New.isnull().sum())
stats.spearmanr(df_New['GLUCOSE'],df_New['Glucose_basal'])

In [ ]:
sns.scatterplot(data =df_New,x = 'Glucose_basal',y='GLUCOSE', sizes = (20,200),legend="full")

In [ ]:
_p=list()
for i in range(len(cor)):
    _p.append(stats.spearmanr(df[cor['x'][i]], df[cor['y'][i]],nan_policy='omit')[1])
cor['p_value']= _p

In [ ]:
cor.drop_duplicates(subset='cor_value',keep='first' ,inplace=True,ignore_index=True)

In [ ]:
cor

In [ ]:
cor.to_csv("all_corr_new.csv")

In [ ]:
ig,axs=plt.subplots(6,4,figsize=(25,40))
axs = axs.ravel()
pd.set_option("display.max_columns",None)
for i in range(len(cor)):
    axs[i].set_title(cor['x'][i]+" AND "+ cor['y'][i])
    sns.scatterplot(data =df,x = cor['x'][i],y=cor['y'][i], ax = axs[i], sizes = (20,200),legend="full", style="GENDER",hue="GENDER",palette="deep",alpha=1)
    sns.despine()

In [ ]:
scatter_matrix(df , figsize=(200,600))

In [ ]:
df['INTERVIE_wgt'].describe()

חילוק הטבלה הגדולה לשתי טבלאות קטנות יותר של נשים וגברים

In [ ]:
male=pd.DataFrame()
female=pd.DataFrame()
male = df.loc[df['GENDER']==1,: ]
male.reset_index(drop=True,inplace=True)
female=df.loc[df['GENDER']==2,: ]
female.reset_index(drop=True,inplace=True)

In [ ]:
female.shape

In [ ]:
male.shape

In [ ]:
male.to_csv('only_male.csv')
female.to_csv('only_female.csv')

In [ ]:
my_report = sv.analyze(female)
my_report.show_html(filepath=r"C:\Users\maxme\Desktop\ביואינפורמטיקה\female_sweetviz.html")

In [ ]:
my_report = sv.analyze(male)
my_report.show_html(filepath=r"C:\Users\maxme\Desktop\ביואינפורמטיקה\male_sweetviz.html")

In [ ]:
my_report = sv.analyze(df)
my_report.show_html(filepath=r"C:\Users\maxme\Desktop\ביואינפורמטיקה\all_sweetviz.html")

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [ ]:
profile.to_file("5.4df.html")

In [ ]:
sns.boxplot(x="GENDER", y="AGE", data=df)
sns.displot(x="AGE", col="GENDER", data=df,rug=True,kind="kde")

In [ ]:
cat = ['GENDER','ETHNICITY','HOUSE_INDIV','FAMILY_SIZE','sw_pregnant','sw_smoke_100','sw_smoke_actual','sw_smoker','sw_smoker_cat','sw_cannabis','last_cannabis_time','sw_drugs','sw_coccaine','sw_heroin','sw_methamphetamine','sw_diab','sw_prediab','sw_diabrisk','sw_insulin','sw_oral_antidiab','diab_retinopathy','insulin_period','CAD','CHF','MI','STROKE','FAMILY_DIAB','sw_asthma_ever','sw_overweight','sw_malignancy','RENAL','DIALYSIS','sw_smoker_cat','pulse_type']
cat_cols=pd.DataFrame()
num_cols=pd.DataFrame()
for i in df.columns:
    if i in cat:
        cat_cols[i]= df[i]
    else:
        num_cols[i]=df[i]


In [ ]:
ig,axs=plt.subplots(50,28,figsize=(200,600))
axs = axs.ravel()
pd.set_option("display.max_columns",None)
k=0
for i in cat_cols.columns:
    for j in num_cols.columns:
            axs[k].set_title(i+" AND "+ j)
            sns.boxplot(x=i, y=j, data=df,ax=axs[k])
            # sns.displot(x=i, col=j, data=df,rug=True,kind="kde")
            sns.despine()
            k+=1
        


In [ ]:
# ig,axs=plt.subplots(55,35,figsize=(200,700))
# axs = axs.ravel()

pd.set_option("display.max_columns",None)
# k=0
for i in cat_cols.columns:
    for j in num_cols.columns:
            # axs[k].set_title(i+" AND "+ j)
            # sns.boxplot(x=i, y=j, data=df,ax=axs[k])
            sns.catplot(x=i, y=j, data=df)
            sns.despine()
            print(i + " "+ j)
            plt.savefig(r"C:\Users\maxme\Desktop\ביואינפורמטיקה\Graphs2\catplot "+ i +" AND "+ j +".png")
            # k+=1
        

In [ ]:
import os.path
pd.set_option("display.max_columns",None)
# k=0
for i in cat_cols.columns:
    for j in num_cols.columns:
        print(i + " "+ j)
        # if os.path.exists(r"C:\Users\maxme\Desktop\ביואינפורמטיקה\Graphs\displot "+ i +" AND "+ j +".png")==False:
                # axs[k].set_title(i+" AND "+ j)
        sns.displot(x=j, col=i, data=df)
                # sns.catplot(x=i, y=j, data=df)
        sns.despine()
                
        plt.savefig(r"C:\Users\maxme\Desktop\ביואינפורמטיקה\Graphs2\displot "+ i +" AND "+ j +".png")
                # k+=1

In [ ]:
df['pulse_type'].value_counts()

In [ ]:

for i in cat_cols.columns:
    for j in num_cols.columns:
            ig, (ax_box, ax_hist) = plt.subplots(58,32, gridspec_kw={"height_ratios": (.15, .85)},sharex=True)
            print(i +' '+j )
            pd.set_option("display.max_columns",None)
            # ax_hist.set_title(i+" AND "+ j)
            # sns.boxplot(x=i, y=j, data=df,ax=axs[k])
            sns.boxplot(x=i, y=j, data=df,ax=ax_box)
            sns.displot(x=j, col=i, data=df,ax=ax_hist)
            sns.despine()
            
            

In [ ]:

# ig, ax= plt.subplots(2,1,figsize=(8,6),
#                       gridspec_kw={"height_ratios": (.40, .85)})
# print(i +' '+j )

            # ax_hist.set_title(i+" AND "+ j)
            # sns.boxplot(x=i, y=j, data=df,ax=axs[k])
# sns.boxplot(x='GENDER', y="AGE", data=df,ax=ax[0] )
# sns.displot(x='AGE',col='GENDER',data=df,ax=ax[1],legend='full', kind="kde",multiple="stack")
# g=sns.jointplot(y='AGE', x="GENDER", data=df,kind = 'hex')
# g = sns.jointplot(data=df, x="GENDER", y="AGE",kind='resid')

g = sns.jointplot(data=df, y='AGE', x='GENDER', height=5, ratio=2, marginal_ticks=True)

g.plot_marginals(sns.displot)
g.plot_joint(sns.boxplot)

In [ ]:
ig,axs=plt.subplots(6,4,figsize=(25,40))
axs = axs.ravel()
pd.set_option("display.max_columns",None)
for i in range(len(cor)):
    if cor['x'][i] in cat_cols.columns:
        axs[i].set_title(cor['x'][i]+" AND "+ cor['y'][i])
        sns.scatterplot(data =df,x = cor['x'][i],y=cor['y'][i], ax = axs[i], sizes = (20,200),legend="full",hue=cor['x'][i])
        sns.despine()
    elif cor['y'][i] in cat_cols.columns:
        axs[i].set_title(cor['x'][i]+" AND "+ cor['y'][i])
        sns.scatterplot(data =df,x = cor['x'][i],y=cor['y'][i], ax = axs[i], sizes = (20,200),legend="full",hue=cor['y'][i])
        sns.despine()
    else:
        axs[i].set_title(cor['x'][i]+" AND "+ cor['y'][i])
        sns.scatterplot(data =df,x = cor['x'][i],y=cor['y'][i], ax = axs[i], sizes = (20,200),legend="full")
        sns.despine()

In [ ]:
ig,axs=plt.subplots(55,35,figsize=(200,700))
axs = axs.ravel()
pd.set_option("display.max_columns",None)
k=0
for i in cat_cols.columns:
    for j in num_cols.columns:
            axs[k].set_title(i+" AND "+ j)
            sns.scatterplot(x=i, y=j, data=df,ax=axs[k],hue='GENDER',style="GENDER",palette="deep",sizes=(20, 200), legend="full")
            # sns.displot(x=i, col=j, data=df,rug=True,kind="kde")
            sns.despine()
            k+=1